## 直线检测算法汇总

在一些任务场景中，通常会用到直线检测算法，比如车道线检测、长度测量等。

主要参考：

1. [](https://mp.weixin.qq.com/s/j6tEg44WRE5BWq0luiEFcw)
2. [直线检测算法汇总](https://blog.csdn.net/WZZ18191171661/article/details/101116949)
3. [直线检测算法博文中缺失的几个源码(Hough_line、LSD、FLD、EDlines、LSWMS、CannyLines、MCMLSD、LSM)](https://aitechtogether.com/article/28878.html)

### Hough_line 直线检测

Hough变换常用于直线检测、圆检测、椭圆检测等。实现步骤如下：

- 创建二维数组或累加器，并将其初始值设置为零
- 用$r$表示行，用$\theta$表示列
- 数组的大小取决于所需要的精度。假设希望角度的精度为1度，则需要180列
- 对于$r$，可能的最大距离是图像的对角线长度。因此取一个像素精度，行数可以是图像的对角线长度

In [ ]:
# coding=utf-8
# 导入相应的python包
import cv2 
import numpy as np 
import os

In [ ]:
# 读取输入图片
img = cv2.imread('../image/lineDetection/line_detection.png') 
# 将彩色图片灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
# 使用Canny边缘检测 
edges = cv2.Canny(gray,50,200,apertureSize = 3) 
# 进行Hough_line直线检测
lines = cv2.HoughLines(edges,1,np.pi/10, 300) 
print(lines)
# 遍历每一个r和theta
for i in range(len(lines)):
    r,theta = lines[i, 0, 0], lines[i, 0, 1]
    # 存储cos(theta)的值
    a = np.cos(theta)
    # 存储sin(theta)的值
    b = np.sin(theta) 
    # 存储rcos(theta)的值
    x0 = a*r 
    # 存储rsin(theta)的值 
    y0 = b*r  
    # 存储(rcos(theta)-1000sin(theta))的值
    x1 = int(x0 + 1000*(-b)) 
    # 存储(rsin(theta)+1000cos(theta))的值
    y1 = int(y0 + 1000*(a)) 
    # 存储(rcos(theta)+1000sin(theta))的值
    x2 = int(x0 - 1000*(-b)) 
    # 存储(rsin(theta)-1000cos(theta))的值
    y2 = int(y0 - 1000*(a))  
    # 绘制直线结果  
    cv2.line(img,(x1,y1), (x2,y2), (0,255,0),2) 
# 保存结果
# cv2.imwrite('test3_r.jpg', img) 
cv2.imshow("result", img)
cv2.waitKey(0)


### HoughLinesP

In [ ]:
# coding=utf-8
# 导入相应的python包
import cv2
import numpy as np

# 读取输入图片
img = cv2.imread('../image/lineDetection/line_detection.png') 
# 将彩色图片灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# 使用Canny边缘检测
edges = cv2.Canny(gray,50,200,apertureSize = 3)
# 进行Hough_line直线检测
lines = cv2.HoughLinesP(edges,1,np.pi/180, 80, 30, 10)

# 遍历每一条直线
for i in range(len(lines)):
	cv2.line(img,(lines[i, 0, 0],lines[i, 0, 1]), (lines[i, 0, 2],lines[i, 0, 3]), (0,255,0),2)
# 保存结果
cv2.imwrite('test3_r.jpg', img)
cv2.imshow("result", img)
cv2.waitKey(0)

### LSD算法

LSD是opencv中集成的一个直线检测算法，该算法的直线检测效果优于Hough算法，而且具有较好的检测速度，推荐使用。LSD快速直线检测算法是由Rafael Grompone、Jeremie Jackbowicz、Jean-Michel Morel于2010年发表在PAMI上的文献《LSD:a Line Segment Dectctor》中提出的，该算法时间复杂度较霍夫变换低。LSD算法通过对图像局部分析，得出直线的像素点集，再通过假设参数进行验证求解，将像素点集合与误差控制集合合并，进而自适应控制误检的数量 。一般来说，要检测图像中的直线，最基本的思想是检测图像中梯度变化较大的像素点集，LSD算法也正是利用梯度信息和行列线（level-line）来进行直线检测的。由于专利问题，opencv已经不能使用lsd算法了。

In [ ]:
# coding=utf-8
import cv2
import numpy as np

# 读取输入图片
img0 = cv2.imread('../image/lineDetection/line_detection.png') 
# 将彩色图片转换为灰度图片
img = cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)

# 创建一个LSD对象
lsd = cv2.createLineSegmentDetector(0)
# 执行检测结果
dlines = lsd.detect(img)
# 绘制检测结果
for dline in dlines[0]:
    x0 = int(round(dline[0][0]))
    y0 = int(round(dline[0][1]))
    x1 = int(round(dline[0][2]))
    y1 = int(round(dline[0][3]))
    cv2.line(img0, (x0, y0), (x1,y1), (0,255,0), 1, cv2.LINE_AA)

# 显示并保存结果
cv2.imwrite('test3_r.jpg', img0)
cv2.imshow("LSD", img0)
cv2.waitKey(0)
cv2.destroyAllWindows()

该算法的检测结果远远优于Hough和HoughP算法；除此之外，上述的检测结果都是使用LSD算法的默认参数进行执行，如果针对特定的参数进行调节，可以取得更好的结果，这种情况一般是在你的特定需求场景中对默认的一些参数进行微调操作，往往能获得意想不到的结果。

### FLD直线检测算法

FLD直线检测算法是在该论文中被引入的，该论文中尝试着使用直线特征来代替原始的SURF点特征进行建筑物识别。与点特征进行相比，线特征具有更容易发现和更好的鲁棒性，线特征基本上不会受到光照、遮挡、视角变化的影响。下面展示了该算法的直线检测效果，从图中我们可以看出，线特征比点特征更好一些。

In [ ]:
# coding=utf-8
import cv2
import numpy as np

# 读取输入图片
img0 = cv2.imread('../image/lineDetection/line_detection.png')
# 将彩色图片转换为灰度图片
img = cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)

# 创建一个LSD对象
fld = cv2.ximgproc.createFastLineDetector()
# 执行检测结果
dlines = fld.detect(img)
# 绘制检测结果
# drawn_img = fld.drawSegments(img0,dlines, )
for dline in dlines:
    x0 = int(round(dline[0][0]))
    y0 = int(round(dline[0][1]))
    x1 = int(round(dline[0][2]))
    y1 = int(round(dline[0][3]))
    cv2.line(img0, (x0, y0), (x1,y1), (0,255,0), 1, cv2.LINE_AA)

# 显示并保存结果
# cv2.imwrite('test3_r.jpg', img0)
cv2.imshow("LSD", img0)
cv2.waitKey(0)
cv2.destroyAllWindows()

### EDlines直线检测算法

EDlines直线检测算法是在该论文中提出的。本文提出了一个快速、无参数的线段检测器，命名为EDLines (Akinlar and Topal, 2011)，它产生强大的和准确的结果，比最快的已知线段检测器速度更快，达到11倍；换句话说，the LSD by Grompone von Gioi et al. (2008a,b, 2010). 我们的探测器还包括一个线的验证步骤定于亥姆霍兹原理Helmholtz principle (Desolneux et al., 2008)，这让它控制错误检测的数量。EDLines得到的结果，我们看到的是，LSD非常相似，有所有主要的线段检测，并有极少数误报。此外， EDLines运行实时以炫目的速度为9.45毫秒，约10倍的速度比LSD对给定的图像。

算法实现步骤：
- 首先给定灰度图，运行新的边缘检测、边缘绘制算法，产生一套干净的，像素相邻的链，称之为边缘
- 利用直线度准则，即最小二乘直线拟合法，从生成的像素链中提取像素
- 线的验证步骤定于亥姆霍兹原理


算法实现见`C++`部分。